# Load checkpoint and imports

In [1]:
import sys
import warnings
warnings.filterwarnings('ignore')
# !{sys.executable} -m pip install seaborn

In [2]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate,train_test_split
import ast
from sklearn.linear_model import LinearRegression,Lasso,LassoCV,MultiTaskLassoCV
from sklearn.metrics import roc_auc_score, mean_squared_error
import scipy
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy import stats
base_path= '../../../../../../T5 EVO/'
from sklearn.preprocessing import StandardScaler
import torch
from sklearn.linear_model import LogisticRegression,LassoCV
import random
# [x[0] for x in os.walk(base_path)]
# input_file_keller_pom = '/local_storage/datasets/farzaneh/openpom/data/curated_datasets/embeddings/pom/keller_pom_embeddings_Apr17.csv'
# 

In [3]:
seed = 2024
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)

In [4]:
times=30
n_components=20

In [5]:
print(os.getcwd())

# Helper Methods

In [6]:
def literal_eval_list(list_string):
    list_string_temp=list_string.values.tolist()
    list_string_all = []
    for value in list_string_temp:
        list_string_all.append(ast.literal_eval(value))
    
    list_numpy = np.asarray(list_string_all)
    return list_numpy

seed=2024
#test
#keller_pom_y=literal_eval_list(keller_pom.y)

In [7]:
# def average_keller(df):
#     df_groupbyCID=df.groupby('CID')[['0.1',
#      '1.1',
#      '2.1',
#      '3.1',
#      '4.1',
#      '5.1',
#      '6.1',
#      '7.1',
#      '8.1',
#      '9.1',
#      '10.1',
#      '11.1',
#      '12.1',
#      '13.1',
#      '14.1',
#      '15.1',
#      '16.1',
#      '17.1',
#      '18.1',
#      '19.1',
#      '20.1',
#      '21.1']].mean().reset_index()

#     df_groupbyCID['y'] = df_groupbyCID.loc[:, '0.1':'21.1'].values.tolist()
#     df_embeddings=df.drop_duplicates(subset=['CID'])
#     df_embeddings=df_embeddings[['CID','embeddings']]
#     df_groupbyCID = pd.merge(df_groupbyCID, df_embeddings, on='CID', how='left')
#     return df_groupbyCID


In [8]:
# def average_sagar(df):
#     df_groupbyCID=df.groupby('CID')[['0.1',
#      '1.1',
#      '2.1',
#      '3.1',
#      '4.1',
#      '5.1',
#      '6.1',
#      '7.1',
#      '8.1',
#      '9.1',
#      '10.1',
#      '11.1',
#      '12.1',
#      '13.1',
#      '14.1'
#      ]].mean().reset_index()

#     df_groupbyCID['y'] = df_groupbyCID.loc[:, '0.1':'14.1'].values.tolist()
#     df_embeddings=df.drop_duplicates(subset=['CID'])
#     df_embeddings=df_embeddings[['CID','embeddings']]
#     df_groupbyCID = pd.merge(df_groupbyCID, df_embeddings, on='CID', how='left')
#     return df_groupbyCID

In [9]:
# def average_keller_subject(df):
#     df_groupbyCID=df.groupby(['CID','subject'])[['0.1',
#      '1.1',
#      '2.1',
#      '3.1',
#      '4.1',
#      '5.1',
#      '6.1',
#      '7.1',
#      '8.1',
#      '9.1',
#      '10.1',
#      '11.1',
#      '12.1',
#      '13.1',
#      '14.1',
#      '15.1',
#      '16.1',
#      '17.1',
#      '18.1',
#      '19.1',
#      '20.1',
#      '21.1']].mean().reset_index()

#     df_groupbyCID['y'] = df_groupbyCID.loc[:, '0.1':'21.1'].values.tolist()
#     df_embeddings=df.drop_duplicates(subset=['CID'])
#     df_embeddings=df_embeddings[['CID','embeddings']]
#     df_groupbyCID = pd.merge(df_groupbyCID, df_embeddings, on='CID', how='left')
#     return df_groupbyCID

In [10]:
# def average_sagar_subject(df):
#     df_groupbyCID=df.groupby(['CID','subject'])[['0.1',
#      '1.1',
#      '2.1',
#      '3.1',
#      '4.1',
#      '5.1',
#      '6.1',
#      '7.1',
#      '8.1',
#      '9.1',
#      '10.1',
#      '11.1',
#      '12.1',
#      '13.1',
#      '14.1'
# ]].mean().reset_index()

#     df_groupbyCID['y'] = df_groupbyCID.loc[:, '0.1':'14.1'].values.tolist()
#     df_embeddings=df.drop_duplicates(subset=['CID'])
#     df_embeddings=df_embeddings[['CID','embeddings']]
#     df_groupbyCID = pd.merge(df_groupbyCID, df_embeddings, on='CID', how='left')
#     return df_groupbyCID

In [7]:
def keep_unique_cids(df):
    
    df = df.drop_duplicates('CID')
    df= df[['CID','embeddings']]
    return df

In [8]:
def custom_linear_regression(X,y):
    # print(y.shape)
    if len(y.shape)>1:
        linreg = MultiTaskLassoCV(max_iter=1000,n_alphas=200,random_state=seed,n_jobs=-1)
    else:
        linreg = LassoCV(max_iter=1000,n_alphas=200,random_state=seed,n_jobs=-1)
    
    estimator= linreg.fit(X,y)
    return estimator

In [9]:
def metrics_per_descritor(X,y,linreg):
    predicted=linreg.predict(X)
    mseerrors = []
    correlations = []
    if len(y.shape)>1:
        for i in range(y.shape[1]):
            mseerror=mean_squared_error(predicted[:,i],y[:,i])
            correlation=scipy.stats.pearsonr(predicted[:,i], y[:,i])
            mseerrors.append(mseerror)
            correlations.append(correlation)
            # print(predicted[:,i], y[:,i])
        
    else:
        mseerror=mean_squared_error(predicted,y)
        correlation=scipy.stats.pearsonr(predicted, y)
        mseerrors.append(mseerror)
        correlations.append(correlation)
     # print(predicted[:,i], y[:,i])
    
    return mseerrors, correlations
        # plot()
        
    
    

In [10]:
# def post_process_results(mserrorrs_corssvalidated,correlations_corssvalidated):
#     mserrorrs_corssvalidated_array = np.asarray(mserrorrs_corssvalidated) 
#     mserrorrs_corssvalidated_mean = mserrorrs_corssvalidated_array.mean(axis=0)
#     mserrorrs_corssvalidated_std = mserrorrs_corssvalidated_array.std(axis=0)
#     # print("correlations_corssvalidated",np.asarray(correlations_corssvalidated).shape)
#     correlations_corssvalidated = np.asarray(correlations_corssvalidated)
#     if len(correlations_corssvalidated.shape)==4:
#         correlations_corssvalidated = np.moveaxis(correlations_corssvalidated, 0,1)
#         # print("correlations_corssvalidateds",correlations_corssvalidated.shape)
#         correlations_corssvalidated = np.squeeze(correlations_corssvalidated,2)
#     # print("correlations_corssvalidatedss",correlations_corssvalidated.shape)
#     statistics_correlations_corssvalidated_array = correlations_corssvalidated[:,:,0]
#     pvalues_correlations_corssvalidated_array = correlations_corssvalidated[:,:,1]
#     statistics_correlations_corssvalidated_mean = statistics_correlations_corssvalidated_array.mean(axis=0)
#     statistics_correlations_corssvalidated_std = statistics_correlations_corssvalidated_array.std(axis=0)

#     return mserrorrs_corssvalidated_mean,mserrorrs_corssvalidated_std,statistics_correlations_corssvalidated_mean,statistics_correlations_corssvalidated_std
    

In [11]:
# shapeeee(22,3)
# (3,22,2)

In [12]:
def post_process_results_df(mserrorrs_corssvalidated,correlations_corssvalidated):
    mserrorrs_corssvalidated_array = np.asarray(mserrorrs_corssvalidated) 
    if len(mserrorrs_corssvalidated_array.shape)==3:
        mserrorrs_corssvalidated_array = np.squeeze(mserrorrs_corssvalidated_array,-1)
        mserrorrs_corssvalidated_array = np.moveaxis(mserrorrs_corssvalidated_array, 0,1)
    # print(mserrorrs_corssvalidated_array.shape,"shapeeee1")
    
    correlations_corssvalidated = np.asarray(correlations_corssvalidated)
    if len(correlations_corssvalidated.shape)==4:
        correlations_corssvalidated = np.moveaxis(correlations_corssvalidated, 0,1)
        # print("correlations_corssvalidateds",correlations_corssvalidated.shape)
        correlations_corssvalidated = np.squeeze(correlations_corssvalidated,2)
    # print(correlations_corssvalidated.shape,"shapeeee2")
    statistics_correlations_corssvalidated_array = correlations_corssvalidated[:,:,0]
    pvalues_correlations_corssvalidated_array = correlations_corssvalidated[:,:,1]

    return mserrorrs_corssvalidated_array,statistics_correlations_corssvalidated_array,pvalues_correlations_corssvalidated_array
    

In [13]:
def pipeline_regression(data_groupbyCID,times,n_components=None,y_i=None):
    #Embeddings are representations extracted from MoLformer or Pom
    # y are chemical features
    mserrorrs_corssvalidated = []
    correlations_corssvalidated = []
    X=np.asarray(data_groupbyCID.embeddings.values.tolist())
    if y_i is not None:
        y=np.asarray(data_groupbyCID.y.values.tolist())[:,y_i].reshape(-1,1)
    else:
       y=np.asarray(data_groupbyCID.y.values.tolist())
    for i in range(times):
       
        # print("min", X.min())
        # print("max", X.max())
          
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed+i)  
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        print(n_components, X_train.shape[1], X_train.shape[0])
        if (n_components is not None) and (n_components < X_train.shape[1]) :
            pca = PCA(n_components=n_components)
            X_train=pca.fit_transform(X_train)
            X_test=pca.transform(X_test)
        linreg =custom_linear_regression(X_train,y_train)
        mseerrors, correlations=metrics_per_descritor(X_test,y_test,linreg)
        mserrorrs_corssvalidated.append(mseerrors)
        correlations_corssvalidated.append(correlations)
    return mserrorrs_corssvalidated, correlations_corssvalidated

In [14]:
def prepare_dataset(ds):
    # ds['y'] = ds['y'].apply(ast.literal_eval)
    ds['embeddings'] = ds['embeddings'].apply(ast.literal_eval)

    # delete y as perception to not mix that 
    if 'y' in ds.columns:
        del ds['y']
    return ds

In [15]:
def select_features(input_file):
    ds_alva = pd.read_csv(input_file)
    ds_alva= ds_alva.rename(columns={"cid":"CID"})
    if 'CID' not in ds_alva:
        ds_alva=ds_alva.rename(columns={"Unnamed: 0": "CID"})
        # ds_alva = ds_alva.reindex(columns=['CID'])
        index_columns = ds_alva.columns.tolist()  # All columns except the last one (NewIndex)

        # ds_alva = ds_alva.set_index(['CID'] + index_columns)

    # if chemical_features_r is None: 
    chemical_features_r=["nCIR",
                     "ZM1", 
                     "GNar", 
                     "S1K", 
                     "piPC08",
                     "MATS1v",
                     "MATS7v",
                     "GATS1v", 
                     "Eig05_AEA(bo)", 
                     "SM02_AEA(bo)",
                     "SM03_AEA(dm)",
                     "SM10_AEA(dm)",
                     "SM13_AEA(dm)",
                      "SpMin3_Bh(v)",
                     "RDF035v",
                     "G1m",
                     "G1v",
                     "G1e",
                     "G3s",
                     "R8u+",
                     "nRCOSR"]

    nonStereoSMILE = list(map(lambda x: "nonStereoSMILES___" + x, chemical_features_r))
    # IsomericSMILES = list(map(lambda x: "IsomericSMILES___" + x, chemical_features_r))
    selected_features = nonStereoSMILE
    features= ['CID','nonStereoSMILES']+selected_features
    print(features)
    ds_alva.set_index(['CID'] + features)
    # print("cc1", ds_alva.columns.values.tolist())
    
    # print("cc2", ds_alva.columns.values.tolist())
    ds_alva_selected = ds_alva[features]
    ds_alva_selected = ds_alva_selected.fillna(0)
    ds_alva_selected['y'] = ds_alva_selected[selected_features].values.tolist()
    return ds_alva_selected,nonStereoSMILE

In [16]:
def min_max_extraction(data_groupbyCID,times,y_i=None):
    min_max_dfs = []
    X=np.asarray(data_groupbyCID.embeddings.values.tolist())
    if y_i is not None:
        y=np.asarray(data_groupbyCID.y.values.tolist())[:,y_i].reshape(-1,1)
    else:
       y=np.asarray(data_groupbyCID.y.values.tolist())
    for i in range(times):
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed+i)  
        # print(X_train.shape,"x shape")
        # print(y_train.shape,"y shape")
        df = pd.DataFrame(y_test)

        # Step 3: Extract the min and max values for each column
        min_values = df.min()
        max_values = df.max()

       # Create DataFrames for min and max values with an additional column for the label
        min_df = pd.DataFrame(min_values).T
        min_df['Type'] = 'Min'
    
        max_df = pd.DataFrame(max_values).T
        max_df['Type'] = 'Max'
    
        # Concatenate the min and max DataFrames
        min_max_df = pd.concat([min_df, max_df])
        min_max_df['Dataset'] = i
    
        # Append the concatenated DataFrame to the lis
    
        # Append the min_max_df to the list
        min_max_dfs.append(min_max_df)
    
    final_df = pd.concat(min_max_dfs)   
        # Step 4: Create a new DataFrame with the min and max values per column
    final_df.set_index(['Dataset', 'Type'], inplace=True)
    # min_max_df = pd.DataFrame([min_values, max_values], index=['Min', 'Max'])
        
    return final_df

In [17]:
def pipeline(model_name,input_file_alva,input_file,times=30,n_components=None,ds="keller",per_descritor=False,count=False):
    # input_file_keller = base_path+'openpom/data/curated_datasets/embeddings/molformer/keller_molformer_embeddings_13_Apr17.csv'
    df=pd.read_csv(input_file)
    df=prepare_dataset(df)
   
    
    # if ds=="keller":
    #     # df_groupbyCID=average_keller(df)
    #     # df_groupbyCIDSubject=average_keller_subject(df)
    #     df_uniqe_cids= keep_unique_cids(df,'0.1','14.1')
    # elif ds=="sagar":
    #     # df_groupbyCID=average_sagar(df)
    #     # df_groupbyCIDSubject=average_sagar_subject(df)
    
    df_uniqe_cids= keep_unique_cids(df)
   
    
    df_alva,features = select_features(input_file_alva)
    
    df_groupbyCID= pd.merge(df_alva,df_uniqe_cids,on="CID")
    # print(len(df_groupbyCID.y.values.tolist()))
    # print(np.asarray(df_groupbyCID.y.values.tolist()).shape)
    

    # print(features)
    # features  =  df_groupbyCID.columns.values.tolist()[1:-2]
    # print("descriptors",descriptors)
    if per_descritor : 
        mserrorrs_df_corssvalidated =[]
        correlations_df_corssvalidated=[]
        # pass
        for i,feature in enumerate(features):
            # print("i",i,feature)
            # print(feature)
            # print(groupbyCID_selected.columns.values.tolist())
            groupbyCID_selected = df_groupbyCID[["CID","y","embeddings",feature]]
            mserrorr_df_corssvalidated, correlation_df_corssvalidated=pipeline_regression(groupbyCID_selected,times=times,n_components=n_components,y_i = i)
            mserrorrs_df_corssvalidated.append(mserrorr_df_corssvalidated)
            # print(mserrorr_df_corssvalidated)
            correlations_df_corssvalidated.append(correlation_df_corssvalidated)
            # print("i",i)
            
    else:
        if count:
            min_max_df=min_max_extraction(df_groupbyCID,times)
            return min_max_df
        else:
        
            mserrorrs_df_corssvalidated, correlations_df_corssvalidated=pipeline_regression(df_groupbyCID,times=times,n_components=n_components)
    # print((mserrorrs_df_corssvalidated),"shape")    
    # mserrorrs_mean_df,mserrorrs_std_df,correlations_mean_df,correlations_std_df =post_process_results(mserrorrs_df_corssvalidated, correlations_df_corssvalidated)
    
    mserrorrs_corssvalidated_df,statistics_correlations_corssvalidated_df,pvalues_correlations_corssvalidated_df=post_process_results_df(mserrorrs_df_corssvalidated, correlations_df_corssvalidated)
    df_df_mse= pd.DataFrame(mserrorrs_corssvalidated_df)
    # df_df_mse = df_df_mse.T
    df_df_mse['model'] = model_name
    df_df_cor= pd.DataFrame(statistics_correlations_corssvalidated_df)
    df_df_cor['model'] = model_name
    # df_keller = pd.concat((df_keller_molformer,df_keller_pom))
    return df_df_mse, df_df_cor

In [28]:
def pipeline_multipleds(model_name,input_file_alvas,input_files,times=30,n_components=None,ds="keller",per_descritor=False,count=False):
    # input_file_keller = base_path+'openpom/data/curated_datasets/embeddings/molformer/keller_molformer_embeddings_13_Apr17.csv'
    df_list = []
    for input_file,input_file_alva in zip(input_files, input_file_alvas):
        print(input_file)
        print(input_file_alva)
        df=pd.read_csv(input_file)
        df=prepare_dataset(df)  
        df_uniqe_cids= keep_unique_cids(df)
        df_alva,features = select_features(input_file_alva)    
        df_groupbyCID= pd.merge(df_alva,df_uniqe_cids,on="CID")
        df_list.append(df_groupbyCID)
    

    result_df = pd.concat(df_list, ignore_index=True)
    result_df = result_df.drop_duplicates('nonStereoSMILES')
    
    if count:
        min_max_df=min_max_extraction(df_groupbyCID,times)
        return min_max_df
    else:
        
        mserrorrs_df_corssvalidated, correlations_df_corssvalidated=pipeline_regression(result_df,times=times,n_components=n_components)
        
    
    mserrorrs_corssvalidated_df,statistics_correlations_corssvalidated_df,pvalues_correlations_corssvalidated_df=post_process_results_df(mserrorrs_df_corssvalidated, correlations_df_corssvalidated)
    df_df_mse= pd.DataFrame(mserrorrs_corssvalidated_df)
    # df_df_mse = df_df_mse.T
    df_df_mse['model'] = model_name
    df_df_cor= pd.DataFrame(statistics_correlations_corssvalidated_df)
    df_df_cor['model'] = model_name
    # df_keller = pd.concat((df_keller_molformer,df_keller_pom))
    return df_df_mse, df_df_cor

In [19]:
# corrs_molfomer,mses_molformer,df_keller_cor_pom,df_keller_mse_pom,df_keller_cor_alva,df_keller_mse_alva =compute_correlation_keller(3 , 2, 2024,per_descritor=True)

In [20]:
def compute_correlation_sagar(times , n_components,per_descritor=False):
    # input_file_sagar_molformer = base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/molformer/sagar_molformer_embeddings_13_Apr17.csv'
    # df_sagar_mse_molformer, df_sagar_cor_molformer = pipeline('molformer',input_file_sagar_molformer,times=times,n_components=n_components,seed=seed,ds="sagar")


    input_file_sagar_alva = base_path+'alignment_olfaction_datasets/data/curated_datasets/alva/sagar_molecules_alva_17Apr.csv'
    
    # df_sagar_mse_alva, df_sagar_cor_alva = pipeline('alva',input_file_sagar_pom,input_file_sagar_alva,times=times,n_components=n_components,ds="sagar")

    
    input_file_sagar_pom = base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/pom/sagar_pom_embeddings_Apr17.csv'
    df_sagar_mse_pom, df_sagar_cor_pom = pipeline('pom',input_file_sagar_alva,input_file_sagar_pom,times=times,n_components=n_components,ds="sagar")
    
    
    
    corrs_molformer=[]

    mses_molformer=[]
    
    for i in [0,1,2,3,4,5,6,7,8,9,10,11,13]:
        input_file_sagar_molformer = base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/molformer/sagar_molformer_embeddings_'+str(i)+'_Apr17.csv'
        df_sagar_mse_molformer, df_sagar_cor_molformer = pipeline('molformer',input_file_sagar_alva,input_file_sagar_molformer,times=times,n_components=n_components,ds="sagar",per_descritor=per_descritor)
        
        
        
        corrs_molformer.append(df_sagar_cor_molformer)
        mses_molformer.append(df_sagar_mse_molformer)

    return corrs_molformer,mses_molformer,df_sagar_cor_pom,df_sagar_mse_pom

In [21]:
def compute_correlation_keller(times , n_components,per_descritor=False):


    input_file_keller_alva = base_path+'alignment_olfaction_datasets/data/curated_datasets/alva/keller_molecules_alva_17Apr.csv'

    
    input_file_keller_pom = base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/pom/keller_pom_embeddings_Apr17.csv'
    df_keller_mse_pom, df_keller_cor_pom = pipeline('pom',input_file_keller_alva,input_file_keller_pom,times=times,n_components=n_components)
    
    
    # df_keller_mse_alva, df_keller_cor_alva = pipeline('alva',input_file_keller_alva,input_file_keller_pom,input_file_keller_alva,times=times,n_components=n_components)
    
    

    
    corrs_molformer=[]

    mses_molformer=[]
    
    for i in [0,1,2,3,4,5,6,7,8,9,10,11,13]:
    # for i in [0,13]:
        input_file_keller_molformer = base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/molformer/keller_molformer_embeddings_'+str(i)+'_Apr17.csv'
        df_keller_mse_molformer, df_keller_cor_molformer = pipeline('molformer',input_file_keller_alva,input_file_keller_molformer,times=times,n_components=n_components,per_descritor=per_descritor)
        
        
        
        corrs_molformer.append(df_keller_cor_molformer)
        mses_molformer.append(df_keller_mse_molformer)

    return corrs_molformer,mses_molformer,df_keller_cor_pom,df_keller_mse_pom

In [22]:
def count_df_x_keller(times ,per_descritor=False,ds="keller"):
    if ds == 'all':
        pipeline_func = pipeline_multipleds
        input_file_alva=[]
        input_file_pom=[]
        
        

        for ds_name in ['snitz','keller','ravia','sagar', 'gslf']:
        # for ds in ['ravia','sagar']:
            input_file_alva.append(base_path+'alignment_olfaction_datasets/data/curated_datasets/alva/'+ds_name+'_molecules_alva_17Apr.csv')
       
            input_file_pom.append(base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/pom/'+ds_name+'_pom_embeddings_Apr17.csv')
    else:
        ds_name = ds
        pipeline_func = pipeline

        input_file_alva = base_path+'alignment_olfaction_datasets/data/curated_datasets/alva/'+ds_name+'_molecules_alva_17Apr.csv'
    
        
        input_file_pom = base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/pom/'+ds_name+'_pom_embeddings_Apr17.csv'

    
    min_max_df = pipeline_func('pom',input_file_alva,input_file_pom,times=times,n_components=n_components,count=True)
    return min_max_df

In [23]:
# df

In [24]:
def compute_correlation(ds,times , n_components,per_descritor=False):

    if ds == 'all':
        pipeline_func = pipeline_multipleds
        input_file_alva=[]
        input_file_pom=[]
        
        

        for ds_name in ['snitz','keller','ravia','sagar', 'gslf']:
        # for ds in ['ravia','sagar']:
            input_file_alva.append(base_path+'alignment_olfaction_datasets/data/curated_datasets/alva/'+ds_name+'_molecules_alva_17Apr.csv')
       
            input_file_pom.append(base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/pom/'+ds_name+'_pom_embeddings_Apr17.csv')
    else:
        ds_name = ds
        pipeline_func = pipeline

        input_file_alva = base_path+'alignment_olfaction_datasets/data/curated_datasets/alva/'+ds_name+'_molecules_alva_17Apr.csv'
    
        
        input_file_pom = base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/pom/'+ds_name+'_pom_embeddings_Apr17.csv'

    
    df_mse_pom, df_cor_pom = pipeline_func('pom',input_file_alva,input_file_pom,times=times,n_components=n_components)
    
    
    # df_keller_mse_alva, df_keller_cor_alva = pipeline('alva',input_file_keller_alva,input_file_keller_pom,input_file_keller_alva,times=times,n_components=n_components)
    
    

    
    corrs_molformer=[]

    mses_molformer=[]
    
    for i in [0,1,2,3,4,5,6,7,8,9,10,11,13]:
    # for i in [0,13]:
        input_file_molformer=[]
        if ds=='all':
            for ds_name in ['keller','ravia','sagar','snitz', 'gslf']:
                input_file_molformer.append(base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/molformer/'+ds_name+'_molformer_embeddings_'+str(i)+'_Apr17.csv')
        else:
            input_file_molformer = base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/molformer/'+ds_name+'_molformer_embeddings_'+str(i)+'_Apr17.csv'
        df_mse_molformer, df_cor_molformer = pipeline_func('molformer',input_file_alva,input_file_molformer,times=times,n_components=n_components,per_descritor=per_descritor)
        
        
        
        corrs_molformer.append(df_cor_molformer)
        mses_molformer.append(df_mse_molformer)

    return corrs_molformer,mses_molformer,df_cor_pom,df_mse_pom

# Extracting Representations

In [25]:
def post_process_tocsv(corrs,tasks):
    corrs[0]["layer"]=0
    corrss = corrs[0]
    for i in range(1,13):
        corrs[i]["layer"] = i
        corrss  = pd.concat([corrss, corrs[i]])
    del corrss['model']
    corrss.columns = tasks+["layer"]    
    corrss['model']='molformer'
    return corrss

In [26]:
chemical_features_r=["nCIR",
                     "ZM1", 
                     "GNar", 
                     "S1K", 
                     "piPC08",
                     "MATS1v",
                     "MATS7v",
                     "GATS1v", 
                     "Eig05_AEA(bo)", 
                     "SM02_AEA(bo)",
                     "SM03_AEA(dm)",
                     "SM10_AEA(dm)",
                     "SM13_AEA(dm)",
                      "SpMin3_Bh(v)",
                     "RDF035v",
                     "G1m",
                     "G1v",
                     "G1e",
                     "G3s",
                     "R8u+",
                     "nRCOSR"]

## Keller

In [29]:
# corrs_molfomer,mses_molformer,df_keller_cor_pom,df_keller_mse_pom =compute_correlation_keller(times , n_components,per_descritor=False)
corrs_molfomer,mses_molformer,df_keller_cor_pom,df_keller_mse_pom =compute_correlation('keller',times , n_components,per_descritor=False)

In [35]:
min_max_df =count_df_x_keller(times ,per_descritor=False,ds="keller")
min_max_df.to_csv('keller_min_max_alva.csv', index=True)   

In [33]:
df_keller_cor_pom.columns = chemical_features_r+["model"]
df_keller_cor_pom.to_csv('df_keller_cor_chemical_pom.csv', index=False)  

df_keller_mse_pom.columns  = chemical_features_r+["model"]
df_keller_mse_pom.to_csv('df_keller_mse_chemical_pom.csv', index=False)  



corrs_molfomer_df = post_process_tocsv(corrs_molfomer,chemical_features_r)
corrs_molfomer_df.to_csv('df_keller_corrs_chemical_molfomer.csv', index=False)   

mses_molformer_df = post_process_tocsv(mses_molformer,chemical_features_r)
mses_molformer_df.to_csv('df_keller_mses_chemical_molfomer.csv', index=False)   

In [34]:
# df=pd.read_csv(input_file)
# df=prepare_dataset(df)

# if ds=="keller":
#     df_groupbyCID=average_keller(df)
#     df_groupbyCIDSubject=average_keller_subject(df)
# elif ds=="sagar":
#     df_groupbyCID=average_sagar(df)
#     df_groupbyCIDSubject=average_sagar_subject(df)

# df_alva,features = select_features(input_file_alva)

## Sagar

In [35]:
corrs_molfomer_sagar,mses_molformer_sagar,df_sagar_cor_pom,df_sagar_mse_pom =compute_correlation('sagar',times , n_components,per_descritor=False)
# corrs_molfomer,mses_molformer,df_keller_cor_pom,df_keller_mse_pom =compute_correlation('keller',times , n_components,per_descritor=False)

In [36]:
min_max_df =count_df_x_keller(times ,per_descritor=False,ds="sagar")
min_max_df.to_csv('sagar_min_max_alva.csv', index=True)   

In [36]:
# corrs_molfomer.to_csv('corrs_molfomer.csv', index=False)  
df_sagar_cor_pom.columns = chemical_features_r+["model"]
df_sagar_cor_pom.to_csv('df_sagar_cor_chemical_pom.csv', index=False)  

df_sagar_mse_pom.columns  = chemical_features_r+["model"]
df_sagar_mse_pom.to_csv('df_sagar_mse_chemical_pom.csv', index=False)  

# df_keller_cor_alva.columns = keller_tasks+["model"]
# df_keller_cor_alva.to_csv('df_keller_cor_alva.csv', index=False)  

# df_keller_mse_alva.columns = keller_tasks+["model"]
# df_keller_mse_alva.to_csv('df_keller_mse_alva.csv', index=False)   


corrs_molfomer_df = post_process_tocsv(corrs_molfomer_sagar,chemical_features_r)
corrs_molfomer_df.to_csv('df_sagar_corrs_chemical_molfomer.csv', index=False)   

mses_molformer_df = post_process_tocsv(mses_molformer_sagar,chemical_features_r)
mses_molformer_df.to_csv('df_sagar_mses_chemical_molfomer.csv', index=False)   

## Ravia

In [37]:
# corrs_molfomer_ravia,mses_molformer_ravia,df_ravia_cor_pom,df_ravia_mse_pom =compute_correlation(times=times , n_components=n_components,per_descritor=False)
corrs_molfomer_ravia,mses_molformer_ravia,df_ravia_cor_pom,df_ravia_mse_pom =compute_correlation('ravia',times , n_components,per_descritor=False)


In [38]:
# corrs_molfomer.to_csv('corrs_molfomer.csv', index=False)  
df_ravia_cor_pom.columns = chemical_features_r+["model"]
df_ravia_cor_pom.to_csv('df_ravia_cor_chemical_pom.csv', index=False)  

df_ravia_mse_pom.columns  = chemical_features_r+["model"]
df_ravia_mse_pom.to_csv('df_ravia_mse_chemical_pom.csv', index=False)  

# df_keller_cor_alva.columns = keller_tasks+["model"]
# df_keller_cor_alva.to_csv('df_keller_cor_alva.csv', index=False)  

# df_keller_mse_alva.columns = keller_tasks+["model"]
# df_keller_mse_alva.to_csv('df_keller_mse_alva.csv', index=False)   


corrs_molfomer_df = post_process_tocsv(corrs_molfomer_ravia,chemical_features_r)
corrs_molfomer_df.to_csv('df_ravia_corrs_chemical_molfomer.csv', index=False)   

mses_molformer_df = post_process_tocsv(mses_molformer_ravia,chemical_features_r)
mses_molformer_df.to_csv('df_ravia_mses_chemical_molfomer.csv', index=False)   

In [37]:
min_max_df =count_df_x_keller(times ,per_descritor=False,ds="ravia")
min_max_df.to_csv('ravia_min_max_alva.csv', index=True)   

## Snitz

In [39]:
# corrs_molfomer_ravia,mses_molformer_ravia,df_ravia_cor_pom,df_ravia_mse_pom =compute_correlation(times=times , n_components=n_components,per_descritor=False)
corrs_molfomer_snitz,mses_molformer_snitz,df_snitz_cor_pom,df_snitz_mse_pom =compute_correlation('snitz',times , n_components,per_descritor=False)


In [40]:
# corrs_molfomer.to_csv('corrs_molfomer.csv', index=False)  
df_snitz_cor_pom.columns = chemical_features_r+["model"]
df_snitz_cor_pom.to_csv('df_snitz_cor_chemical_pom.csv', index=False)  

df_snitz_mse_pom.columns  = chemical_features_r+["model"]
df_snitz_mse_pom.to_csv('df_snitz_mse_chemical_pom.csv', index=False)  


corrs_molfomer_df = post_process_tocsv(corrs_molfomer_snitz,chemical_features_r)
corrs_molfomer_df.to_csv('df_snitz_corrs_chemical_molfomer.csv', index=False)   

mses_molformer_df = post_process_tocsv(mses_molformer_snitz,chemical_features_r)
mses_molformer_df.to_csv('df_snitz_mses_chemical_molfomer.csv', index=False)   

In [38]:
min_max_df =count_df_x_keller(times ,per_descritor=False,ds="snitz")
min_max_df.to_csv('snitz_min_max_alva.csv', index=True)   

## GS-LF

In [ ]:
# ds='gslf'
# input_file_alva = base_path+'alignment_olfaction_datasets/data/curated_datasets/alva/'+ds+'_molecules_alva_17Apr.csv'
# # input_file_pom = base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/pom/'+ds+'_pom_embeddings_Apr17.csv'
# alva = pd.read_csv(input_file_alva)
# alva
# # # pom.columns.values.tolist()
# # # pom[pom['CID']]



In [33]:
# input_file_alva=base_path+'alignment_olfaction_datasets/data/curated_datasets/alva/'+'gslf'+'_molecules_alva_17Apr.csv'


# select_features(input_file_alva)[0]
# ds_alva

# ds_alva=ds_alva.rename(columns={"Unnamed: 0": "CID"})
# ds_alva
# # ds_alva = ds_alva.reindex(columns=['CID'])
# index_columns = ds_alva.columns.tolist()  # All columns except the last one (NewIndex
# index_columns
# # Setting the new multi-level index
# ds_alva = ds_alva.set_index(index_columns)

# # input_file_pom.append(base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/pom/'+ds+'_pom_embeddings_Apr17.csv')
# # pd.read_csv(input_file_pom[0])

In [57]:
corrs_molfomer_gslf,mses_molformer_gslf,df_gslf_cor_pom,df_gslf_mse_pom =compute_correlation('gslf',times , n_components,per_descritor=False)

In [35]:
# corrs_molfomer.to_csv('corrs_molfomer.csv', index=False)  
df_gslf_cor_pom.columns = chemical_features_r+["model"]
df_gslf_cor_pom.to_csv('df_gslf_cor_chemical_pom.csv', index=False)  

df_gslf_mse_pom.columns  = chemical_features_r+["model"]
df_gslf_mse_pom.to_csv('df_gslf_mse_chemical_pom.csv', index=False)  


corrs_molfomer_df = post_process_tocsv(corrs_molfomer_gslf,chemical_features_r)
corrs_molfomer_df.to_csv('df_gslf_corrs_chemical_molfomer.csv', index=False)   

mses_molformer_df = post_process_tocsv(mses_molformer_gslf,chemical_features_r)
mses_molformer_df.to_csv('df_gslf_mses_chemical_molfomer.csv', index=False)   

In [39]:
min_max_df =count_df_x_keller(times ,per_descritor=False,ds="gslf")
min_max_df.to_csv('gslf_min_max_alva.csv', index=True)   

## Combination

In [50]:
# # pd

# input_file_alvas=[]
# input_files=[]
# # for ds in ['sagar','snitz','keller','ravia', 'gslf']:
#         # for ds in ['ravia','sagar']:

# ds='sagar'
# input_file_alva = base_path+'alignment_olfaction_datasets/data/curated_datasets/alva/'+ds+'_molecules_alva_17Apr.csv'
       
# input_file = base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/pom/'+ds+'_pom_embeddings_Apr17.csv'

# # for input_file,input_file_alva in zip(input_files, input_file_alvas):
# # print(input_file)
# # print(input_file_alva)
# df=pd.read_csv(input_file)
# df=prepare_dataset(df)  
# df_uniqe_cids= keep_unique_cids(df)
# df_alva,features = select_features(input_file_alva)    
# # print(df_alva)
# df_groupbyCID= pd.merge(df_alva,df_uniqe_cids,on="CID")
# df_list.append(df_groupbyCID)

In [29]:
min_max_df =count_df_x_keller(times ,per_descritor=False,ds="all")
min_max_df.to_csv('all_min_max_alva.csv', index=True)   

In [61]:
corrs_molfomer_all,mses_molformer_all,df_all_cor_pom,df_all_mse_pom =compute_correlation('all',times , n_components,per_descritor=False)

In [62]:
df_all_cor_pom.columns = chemical_features_r+["model"]
df_all_cor_pom.to_csv('df_all_cor_chemical_pom.csv', index=False)  

df_all_mse_pom.columns  = chemical_features_r+["model"]
df_all_mse_pom.to_csv('df_all_mse_chemical_pom.csv', index=False)  


corrs_molfomer_df = post_process_tocsv(corrs_molfomer_all,chemical_features_r)
corrs_molfomer_df.to_csv('df_all_corrs_chemical_molfomer.csv', index=False)   

mses_molformer_df = post_process_tocsv(mses_molformer_all,chemical_features_r)
mses_molformer_df.to_csv('df_all_mses_chemical_molfomer.csv', index=False)   